In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [12]:
pd.options.display.max_rows = 10
df = pd.read_csv('fake_job_postings.csv')
df

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,17876,Account Director - Distribution,"CA, ON, Toronto",Sales,NaN,Vend is looking for some awesome new talent to...,Just in case this is the first time you’ve vis...,To ace this role you:Will eat comprehensive St...,What can you expect from us?We have an open cu...,0,1,1,Full-time,Mid-Senior level,NaN,Computer Software,Sales,0
17876,17877,Payroll Accountant,"US, PA, Philadelphia",Accounting,NaN,WebLinc is the e-commerce platform and service...,The Payroll Accountant will focus primarily on...,- B.A. or B.S. in Accounting- Desire to have f...,Health &amp; WellnessMedical planPrescription ...,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Internet,Accounting/Auditing,0
17877,17878,Project Cost Control Staff Engineer - Cost Con...,"US, TX, Houston",NaN,NaN,We Provide Full Time Permanent Positions for m...,Experienced Project Cost Control Staff Enginee...,At least 12 years professional experience.Abil...,NaN,0,0,0,Full-time,NaN,NaN,NaN,NaN,0
17878,17879,Graphic Designer,"NG, LA, Lagos",NaN,NaN,NaN,Nemsia Studios is looking for an experienced v...,1. Must be fluent in the latest versions of Co...,Competitive salary (compensation will be based...,0,0,1,Contract,Not Applicable,Professional,Graphic Design,Design,0


# Remove Undesired Columns  and fill nulls with " " 

In [13]:
df.drop(['job_id','salary_range'] , axis = 1 , inplace=True)
df.fillna(' ' , inplace=True)

In [14]:
text_cols = list(df.select_dtypes(include = ['object']).columns)
text_cols

['title',
 'location',
 'department',
 'company_profile',
 'description',
 'requirements',
 'benefits',
 'employment_type',
 'required_experience',
 'required_education',
 'industry',
 'function']

In [17]:
df['Text'] = df[text_cols[0]] + " " + df[text_cols[1]] + " " + df[text_cols[2]] + " " + df[text_cols[3]] + " " + df[text_cols[4]] + " " + df[text_cols[5]] + " " +df[text_cols[6]] + " " + df[text_cols[7]] + " " +df[text_cols[8]] + " " + df[text_cols[9]] + " " +df[text_cols[10]] + " " + df[text_cols[11]]

In [19]:
df.drop(text_cols , inplace = True , axis = 1)

In [20]:
df

,telecommuting,has_company_logo,has_questions,fraudulent,Text
0,0,1,0,0,"Marketing Intern US, NY, New York Marketing We..."
1,0,1,0,0,"Customer Service - Cloud Video Production NZ, ..."
2,0,1,0,0,"Commissioning Machinery Assistant (CMA) US, IA..."
3,0,1,0,0,"Account Executive - Washington DC US, DC, Wash..."
4,0,1,1,0,"Bill Review Manager US, FL, Fort Worth SpotS..."
...,...,...,...,...,...
17875,0,1,1,0,"Account Director - Distribution CA, ON, Toron..."
17876,0,1,1,0,"Payroll Accountant US, PA, Philadelphia Accoun..."
17877,0,0,0,0,Project Cost Control Staff Engineer - Cost Con...
17878,0,0,1,0,"Graphic Designer NG, LA, Lagos Nemsia Stud..."


# Before Anything

In [21]:
df.at[0,'Text']

"Marketing Intern US, NY, New York Marketing We're Food52, and we've created a groundbreaking and award-winning cooking site. We support, connect, and celebrate home cooks, and give them everything they need in one place.We have a top editorial, business, and engineering team. We're focused on using technology to find new and better ways to connect people around their specific food interests, and to offer them superb, highly curated information about food and cooking. We attract the most talented home cooks and contributors in the country; we also publish well-known professionals like Mario Batali, Gwyneth Paltrow, and Danny Meyer. And we have partnerships with Whole Foods Market and Random House.Food52 has been named the best food website by the James Beard Foundation and IACP, and has been featured in the New York Times, NPR, Pando Daily, TechCrunch, and on the Today Show.We're located in Chelsea, in New York City. Food52, a fast-growing, James Beard Award-winning online food communi

# Text Preproceccing Class

In [24]:
import re
from nltk.corpus import stopwords 
import string 
from nltk.stem import WordNetLemmatizer
punc = string.punctuation
lmt = WordNetLemmatizer()
s_words = stopwords.words('english')

In [37]:
class TextPreprocessing() :
  
        
    def Clean(self , df) :
        self.df = df
        df_copy = df.copy(deep = True)
        text_cols = list(df_copy.select_dtypes(include = ['object']).columns)
        for col in text_cols :
            for idx, text in enumerate(df_copy[col]) :
                te = []
                word = re.sub(r'(@|#)\w+' , '' , text)
                word = re.sub("[,.]", "", word)
                word = re.sub(r'https?://\S+' , '' , word)
                word = re.sub(r'(\?|!)+' , '' , word)
                word = re.sub(r"\(|\)", "", word)
                word = re.sub(r'(^\s+)' , '' , word)
                word = re.sub(r'(\s+$)' , '' , word)
                word = re.sub(r'\d+' , '' , word)
                word = word.split()
                for i in word :
                    if (i not in s_words) & (i not in punc) :
                        i = i.lower()
                        i = lmt.lemmatize(i , 'v')
                        te.append(i)
                df_copy.at[idx , col] = te
        return df_copy
    
    def Vactorize (self, df , target_name) :
        self.df = df
        self.target_name = target_name
        df_cleaned = df.copy(deep = True)
        text_cols = list(df_cleaned.select_dtypes(include = ['object']).columns)
        pos_word = {}
        neg_word = {}

        pos_df = df_cleaned[df_cleaned[target_name] == 1].reset_index(drop=True)
        neg_df = df_cleaned[df_cleaned[target_name] == 0].reset_index(drop=True)
        
        for col in text_cols :

            pos_word[col] = [word for sublist in pos_df[col] for word in sublist]
            neg_word[col] = [word for sublist in neg_df[col] for word in sublist]



        pos_freq = {}
        neg_freq = {}
        for key in pos_word.keys() :
            positive_dict = {}
            for word in pos_word[key] :
                positive_dict[word] = positive_dict.get(word , 0) + 1

            pos_freq[key] = positive_dict


        for key in neg_word.keys() :
            negative_dict = {}
            for word in neg_word[key] :
                negative_dict[word] = negative_dict.get(word , 0) + 1

            neg_freq[key] = negative_dict
            
        return pos_freq , neg_freq

        
        
        
    def Vactorization (self , df , target_name) :
        self.df = df
        self.target_name = target_name
        df_cleaned = df.copy(deep = True)
        text_cols = list(df_cleaned.select_dtypes(include = ['object']).columns)
        pos_freq , neg_freq = TextPreprocessing().Vactorize(df_cleaned , target_name)

        for col in text_cols :
            df_cleaned['{}_pos'.format(col)] = 0
            df_cleaned['{}_neg'.format(col)] = 0
            for idx, List in enumerate(df_cleaned[col]) :
                pos_frequent = 0
                neg_frequent = 0
                for word in List :
                    pos_frequent += pos_freq[col].get(word , 0)
                    neg_frequent += neg_freq[col].get(word , 0)



                df_cleaned.at[idx ,'{}_pos'.format(col)] = pos_frequent
                df_cleaned.at[idx ,'{}_neg'.format(col)] = neg_frequent
            df_cleaned.drop([col] , axis = 1 , inplace = True)
        return df_cleaned
    
    
    def fit_transform(self , df , target_name) :
        self.df = df
        self.target_name = target_name
        
        df_cleaned = TextPreprocessing().Clean(df)
        df_vact = TextPreprocessing().Vactorization(df_cleaned , target_name)
        
        return df_vact
    
    
    def Naive_Bayes(self , df, target_name) :
        self.df = df
        self.target_name = target_name
        df_naive = TextPreprocessing().Clean(df)
        pos_freq , neg_freq = TextPreprocessing().Vactorize(df_naive , target_name)
        text_cols = list(df_naive.select_dtypes(include = ['object']).columns)
        
        v_n_pos , v_n_neg = {} , {}

        for key in pos_freq.keys() :
            v_n_pos[key] = len(pos_freq[key])
            n = 0
            for word in pos_freq[key] :
                n += pos_freq[key].get(word , 0)
            v_n_pos[key]+=n
        for key in neg_freq.keys() :
            v_n_neg[key] = len(neg_freq[key])
            n = 0
            for word in neg_freq[key] :
                n += neg_freq[key].get(word , 0)
            v_n_neg[key]+=n
            
            
        prob_pos_dict = {}
        for key in pos_freq.keys():
            positive_dict = {}
            for word in pos_freq[key] :
                positive_dict[word] = (pos_freq[key].get(word , 0) + 1) / (v_n_pos[key])
            prob_pos_dict[key] = positive_dict




        prob_neg_dict = {}
        for key in neg_freq.keys():
            negative_dict = {}
            for word in neg_freq[key] :
                negative_dict[word] = (neg_freq[key].get(word , 0) + 1) / (v_n_neg[key])
            prob_neg_dict[key] = negative_dict
            
            
            
        for col in text_cols :
            df_naive['{}_probs'.format(col)] = 0
            for idx, List in enumerate(df_naive[col]) :
                score = 0
                for word in List :
                    try :
                        b = np.log((prob_pos_dict[col].get(word , 0)) / (prob_neg_dict[col].get(word , 0)))
                        if b == -float('inf') :
                            pass
                        else :
                            score +=b
                    except :
                        pass

                df_naive['{}_probs'.format(col)][idx] = score
            df_naive.drop([col] , axis = 1 , inplace = True)
            
            
        return {'probs_pos':prob_pos_dict ,'probs_neg':prob_neg_dict } , df_naive        

In [38]:
pre = TextPreprocessing()
df_cleaned = pre.Clean(df)

# After Cleaning

In [39]:
df_cleaned

,telecommuting,has_company_logo,has_questions,fraudulent,Text
0,0,1,0,0,"[market, intern, us, ny, new, york, market, we..."
1,0,1,0,0,"[customer, service, cloud, video, production, ..."
2,0,1,0,0,"[commission, machinery, assistant, cma, us, ia..."
3,0,1,0,0,"[account, executive, washington, dc, us, dc, w..."
4,0,1,1,0,"[bill, review, manager, us, fl, fort, worth, s..."
...,...,...,...,...,...
17875,0,1,1,0,"[account, director, distribution, ca, on, toro..."
17876,0,1,1,0,"[payroll, accountant, us, pa, philadelphia, ac..."
17877,0,0,0,0,"[project, cost, control, staff, engineer, cost..."
17878,0,0,1,0,"[graphic, designer, ng, la, lagos, nemsia, stu..."


In [40]:
df_cleaned.at[0,'Text']

['market',
 'intern',
 'us',
 'ny',
 'new',
 'york',
 'market',
 "we're",
 'food',
 "we've",
 'create',
 'groundbreaking',
 'award-winning',
 'cook',
 'site',
 'we',
 'support',
 'connect',
 'celebrate',
 'home',
 'cook',
 'give',
 'everything',
 'need',
 'one',
 'placewe',
 'top',
 'editorial',
 'business',
 'engineer',
 'team',
 "we're",
 'focus',
 'use',
 'technology',
 'find',
 'new',
 'better',
 'ways',
 'connect',
 'people',
 'around',
 'specific',
 'food',
 'interest',
 'offer',
 'superb',
 'highly',
 'curated',
 'information',
 'food',
 'cook',
 'we',
 'attract',
 'talented',
 'home',
 'cook',
 'contributors',
 'country;',
 'also',
 'publish',
 'well-known',
 'professionals',
 'like',
 'mario',
 'batali',
 'gwyneth',
 'paltrow',
 'danny',
 'meyer',
 'and',
 'partnerships',
 'whole',
 'foods',
 'market',
 'random',
 'housefood',
 'name',
 'best',
 'food',
 'website',
 'jam',
 'beard',
 'foundation',
 'iacp',
 'feature',
 'new',
 'york',
 'time',
 'npr',
 'pando',
 'daily',
 'tec

# Vactorization 

In [31]:
df_vatorize = pre.Vactorization(df_cleaned , target_name='fraudulent')
df_vatorize

,telecommuting,has_company_logo,has_questions,fraudulent,Text_pos,Text_neg
0,0,1,0,0,42977,1134099
1,0,1,0,0,121873,3145546
2,0,1,0,0,47002,987573
3,0,1,0,0,88584,2314156
4,0,1,1,0,64453,1517040
...,...,...,...,...,...,...
17875,0,1,1,0,117316,3276941
17876,0,1,1,0,84300,2054748
17877,0,0,0,0,74684,1611543
17878,0,0,1,0,26870,676917


# Naieve Bayes

In [34]:
proba , df_naieve = pre.Naive_Bayes(df , target_name='fraudulent')
df_naieve

,telecommuting,has_company_logo,has_questions,fraudulent,Text_probs
0,0,1,0,0,-56.762489
1,0,1,0,0,-148.539290
2,0,1,0,0,23.242880
3,0,1,0,0,-70.155070
4,0,1,1,0,4.594954
...,...,...,...,...,...
17875,0,1,1,0,-227.040812
17876,0,1,1,0,-16.792167
17877,0,0,0,0,25.274041
17878,0,0,1,0,-24.926743
